In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [21]:
import urllib2
import json
import glob

## Store the names of the json files in a list

In [3]:
filename = ["85.json", "118434.json"]

### To automate finding json files: use `glob` method in python

In [23]:
file_paths = []
for paths in glob.glob("*.json"):
    file_paths.append(paths)

print file_paths

['118434.json', '85.json']


### Skip to [functions](#Functions-for-automating-scraping) to check the functions and get datasets

## Load JSON file

In [ ]:
with open(filename) as json_file:
    content = json.load(json_file)

In [ ]:
content

{u'absolute_url': u'/opinion/85/morton-v-hall/',
 u'author': None,
 u'author_str': u'',
 u'cluster': u'http://www.courtlistener.com/api/rest/v3/clusters/85/',
 u'date_created': u'2010-03-13T23:57:55Z',
 u'date_modified': u'2017-03-23T23:37:38.640459Z',
 u'download_url': u'http://www.ca9.uscourts.gov/datastore/opinions/2010/03/12/07-55574.pdf',
 u'extracted_by_ocr': False,
 u'html': u'',
 u'html_columbia': None,
 u'html_lawbox': u'<div>\n<center><b>599 F.3d 942 (2010)</b></center>\n<center><h1>Bruce Alan MORTON, Plaintiff-Appellant,<br>\nv.<br>\nJames HALL, e/s/a Jim Hall; Vincent Anderson; Christy Hale; Richard Howell; Albert Luna, Defendants-Appellees.</h1></center>\n<center>No. 07-55574.</center>\n<center><p><b>United States Court of Appeals, Ninth Circuit.</b></p></center>\n<center>Submitted March 1, 2010.<sup>[*]</sup></center>\n<center>Filed March 12, 2010.</center>\n<p><span class="star-pagination">*943</span> Konrad L. Trope, Novo Law Group, P.C., Los Angeles, CA, for the plaint

## Get html content from the JSON

In [ ]:
citation_html = content.get("html_with_citations")

In [ ]:
citation_html

u'<div>\n<center><b><span class="citation no-link"><span class="volume">599</span> <span class="reporter">F.3d</span> <span class="page">942</span></span> (2010)</b></center>\n<center><h1>Bruce Alan MORTON, Plaintiff-Appellant,<br>\nv.<br>\nJames HALL, e/s/a Jim Hall; Vincent Anderson; Christy Hale; Richard Howell; Albert Luna, Defendants-Appellees.</h1></center>\n<center>No. 07-55574.</center>\n<center><p><b>United States Court of Appeals, Ninth Circuit.</b></p></center>\n<center>Submitted March 1, 2010.<sup>[*]</sup></center>\n<center>Filed March 12, 2010.</center>\n<p><span class="star-pagination">*943</span> Konrad L. Trope, Novo Law Group, P.C., Los Angeles, CA, for the plaintiff-appellant.</p>\n<p>Kenneth G. Lake, Deputy Attorney General for the State of California, Los Angeles, CA, for the defendants-appellees.</p>\n<p>Before: RONALD M. GOULD and SANDRA S. IKUTA, Circuit Judges, and LLOYD D. GEORGE, District Judge.<sup>[**]</sup></p>\n<p>GOULD, Circuit Judge:</p>\n<p>Plaintiff-A

In [ ]:
soup = BeautifulSoup(citation_html, 'html.parser')

In [ ]:
print soup.prettify()

## Find all the **center** tags to get `case_id`, `case_name`

In [ ]:
center_content = soup.find_all("center")

In [ ]:
center_content

[<center><b><span class="citation no-link"><span class="volume">599</span> <span class="reporter">F.3d</span> <span class="page">942</span></span> (2010)</b></center>,
 <center><h1>Bruce Alan MORTON, Plaintiff-Appellant,<br/>\nv.<br/>\nJames HALL, e/s/a Jim Hall; Vincent Anderson; Christy Hale; Richard Howell; Albert Luna, Defendants-Appellees.</h1></center>,
 <center>No. 07-55574.</center>,
 <center><p><b>United States Court of Appeals, Ninth Circuit.</b></p></center>,
 <center>Submitted March 1, 2010.<sup>[*]</sup></center>,
 <center>Filed March 12, 2010.</center>]

In [ ]:
case_id = []
for lines in center_content:
    # print lines.get_text()
    try:
        if "No." in lines.get_text():
            print lines
            case_id.append(lines.get_text().encode("utf-8"))
    except:
        case_id.append("Not Found")

<center>No. 07-55574.</center>


In [ ]:
case_id

['No. 07-55574.']

In [ ]:
case_name = []
for lines in center_content:
    try:
        h1_tags = lines.find("h1")
        if h1_tags:
            print h1_tags.get_text()
            names = h1_tags.get_text().encode("utf-8")
            case_name.append(names)
    except:
        print "Some Error"

Bruce Alan MORTON, Plaintiff-Appellant,
v.
James HALL, e/s/a Jim Hall; Vincent Anderson; Christy Hale; Richard Howell; Albert Luna, Defendants-Appellees.


In [ ]:
case_name

['Bruce Alan MORTON, Plaintiff-Appellant,\nv.\nJames HALL, e/s/a Jim Hall; Vincent Anderson; Christy Hale; Richard Howell; Albert Luna, Defendants-Appellees.']

In [ ]:
current_doc_citation = []
for lines in center_content:
    try:
        citation_volume = lines.find(class_="volume")
        print citation_volume.get_text()
        citations = citation_volume.get_text().encode("utf-8")
        current_doc_citation.append(citations)
    except:
        print "Oops!"

599
Oops!
Oops!
Oops!
Oops!
Oops!


In [ ]:
current_doc_citation

['599']

## Find all the class tags with value='citation' to get the cited document `id`, `link`, `name`

In [ ]:
opinions_cited_id = []
opinions_cited_links = []

citation_tags = soup.find_all(class_='citation')

for lines in citation_tags:
    #print lines
    try:
        data_id = lines["data-id"].encode("utf-8")
        #print data_id
        links = lines.a['href'].encode("utf-8")
        
        opinions_cited_id.append(data_id)
        opinions_cited_links.append(links)
    except:
        print "error"

# Include only unique ids in the list
opinions_cited_id = list(set(opinions_cited_id))
opinions_cited_links = list(set(opinions_cited_links))

error
error


In [ ]:
print opinions_cited_id
print opinions_cited_links

['781335', '145763', '765065', '780452', '118434', '1433946']
['/opinion/765065/macarthur-rumbles-v-donald-r-hill-black-and-beckett/', '/opinion/118434/booth-v-churner/', '/opinion/145763/jones-v-bock/', '/opinion/780452/earl-wayne-wyatt-v-cal-a-terhune-suzan-hubbard-warden/', '/opinion/781335/burlington-northern-santa-fe-railroad-company-a-delaware-corporation-v/', '/opinion/1433946/griffin-v-arpaio/']


In [ ]:
# opinions_cited_names = []
# citation_names = soup.find_all("i")
# for contents in citation_names:
#     names = contents.get_text().encode("utf-8")
#     print names.lower()
#     for links in opinions_cited_links:
#         if names.lower() in links:
#             print names
#             opinions_cited_names.append(names)

see
see
griffin v. arpaio,
burlington n. santa fe r.r. co. v. assiniboine & sioux tribes of the fort peck reservation,
booth v. churner,
id.
booth
Booth
wyatt v. terhune,
jones v. bock,
griffin,
id.
see
rumbles v. hill,
lunsford v. jumao-as,


In [ ]:
# Store them as csv
dataset_dict = {}
dataset_dict["Case_ID"] = case_id
dataset_dict["Case_Name"] = case_name
dataset_dict["Opinions_cited"] = opinions_cited_id
dataset_dict["Opinions_links"] = opinions_cited_links

In [ ]:
dataset_dict

{'Case_ID': ['No. 07-55574.'],
 'Case_Name': ['Bruce Alan MORTON, Plaintiff-Appellant,\nv.\nJames HALL, e/s/a Jim Hall; Vincent Anderson; Christy Hale; Richard Howell; Albert Luna, Defendants-Appellees.'],
 'Opinions_cited': ['781335',
  '145763',
  '765065',
  '780452',
  '118434',
  '1433946'],
 'Opinions_links': ['/opinion/765065/macarthur-rumbles-v-donald-r-hill-black-and-beckett/',
  '/opinion/118434/booth-v-churner/',
  '/opinion/145763/jones-v-bock/',
  '/opinion/780452/earl-wayne-wyatt-v-cal-a-terhune-suzan-hubbard-warden/',
  '/opinion/781335/burlington-northern-santa-fe-railroad-company-a-delaware-corporation-v/',
  '/opinion/1433946/griffin-v-arpaio/']}

## Storing as JSON

In [ ]:
with open("dataset.json", 'w') as js_file:
    json.dump(dataset_dict, js_file, indent=4, sort_keys=True)

## Functions for automating scraping

In [10]:
def open_json(filename):
    with open(filename) as json_file:
        content = json.load(json_file)
  
    return content


In [11]:
def get_html_content(json_content):
    html_content = json_content.get("html_with_citations")
    soup = BeautifulSoup(html_content, 'html.parser')
  
    return soup

In [13]:
def get_case_details(soup):
  # Get center tags for case id and case names
  center_tags = soup.find_all("center")
  
  for lines in center_tags:
    # print lines.get_text()
    try:
        if "No." in lines.get_text():
            # print lines
            case_id = lines.get_text().encode("utf-8")
        
        h1_tags = lines.find("h1") # For case names
        if h1_tags:
            # print h1_tags.get_text()
            names = h1_tags.get_text().encode("utf-8")
            case_name = names
    except:
        case_id = "Not Found"
  
  return case_id, case_name

In [7]:
def get_citation_details(soup):
  
    opinions_cited_id = []
    opinions_cited_links = []

    citation_tags = soup.find_all(class_='citation')

    for lines in citation_tags:
      #print lines
        try:
            data_id = lines["data-id"].encode("utf-8")
            links = lines.a['href'].encode("utf-8")

            opinions_cited_id.append(data_id)
            opinions_cited_links.append(links)
        except:
            print "error"

    # Include only unique ids in the list
    opinions_cited_id = list(set(opinions_cited_id))
    opinions_cited_links = list(set(opinions_cited_links))

    return opinions_cited_id, opinions_cited_links

## Use the below code to store scraped data as CSV

In [15]:
dataset_dict = {}

case_id = []
case_name = []
opinion_id = []
opinion_link = []

for files in filename:
    print "Extracting file : {}".format(files)
    content = open_json(files)
    soup = get_html_content(content)
    case_id_str, case_name_str = get_case_details(soup)
    case_id.append(case_id_str)
    case_name.append(case_name_str)
  
    opinions_cited_id, opinions_cited_links = get_citation_details(soup)

    opinion_id.append(opinions_cited_id)
    opinion_link.append(opinions_cited_links)

# Store to dict
dataset_dict["Case_ID"] = case_id
dataset_dict["Case_Name"] = case_name
dataset_dict["Opinions_cited"] = opinion_id
dataset_dict["Opinions_links"] = opinion_link
  
# Convert to DataFrame
df = pd.DataFrame(dataset_dict)

# Store as csv
df.to_csv("dataset.csv", index=None)

Extracting file : 85.json
error
error
Extracting file : 118434.json
error
error
error
error
error
error
error
error
error
error


## Use the below code to store the scraped data as JSON

In [20]:
dataset_list = []

for files in filename:
    print "Extracting file : {}".format(files)
    content = open_json(files)
    soup = get_html_content(content)
    case_id_str, case_name_str = get_case_details(soup)
    # case_id.append(case_id_str)
    # case_name.append(case_name_str)
  
    opinions_cited_id, opinions_cited_links = get_citation_details(soup)

    # opinion_id.append(opinions_cited_id)
    # opinion_link.append(opinions_cited_links)

    # Store to dict
    dataset_dict = {}
    dataset_dict["Case_ID"] = case_id_str
    dataset_dict["Case_Name"] = case_name_str
    dataset_dict["Opinions_cited"] = opinions_cited_id
    dataset_dict["Opinions_links"] = opinions_cited_links
  
    dataset_list.append(dataset_dict)

# Store as JSON
with open("dataset.json", 'w') as js_file:
    json.dump(dataset_list, js_file, indent=4, sort_keys=True)

Extracting file : 85.json
error
error
Extracting file : 118434.json
error
error
error
error
error
error
error
error
error
error
